# Data voorbewerking

De data voor dit project is afkomstig van het back office systeem van Financial Markets, deels aangevuld met extra data vanuit Bloomberg.
De data bestaat uit de volgende elementen:

- Bond data
- Bondprijzen
- Government Yield curves
- Inflation data

Alle data is extracted en opgeslagen in csv files. In dit workbook lopen we door de data voorbereiding heen. Alle hier genoemde stappen kunnen ook geautomatiseerd worden uitgevoerd door het shell command 'Make Data'.

In [8]:
%load_ext autoreload
%autoreload 2

import sys
import pandas as pd

sys.path.insert(0, "..") 
from src.data import make_dataset
from src.features import build_features

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Bond data

In [9]:

# Get bond data, drop unneeded columns, convert formats and strip training blanks
df_bonds = make_dataset.get_bond_data()

2022-01-04 15:25:53.947 | INFO     | src.data.make_dataset:get_bond_data:41 - Load bond data
2022-01-04 15:25:53.948 | INFO     | src.data.make_dataset:read_csv:26 - Loading data from ..\data\raw\bonds.csv


In [10]:
df_bonds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   ccy                225 non-null    string        
 1   country            225 non-null    string        
 2   issue_dt           225 non-null    datetime64[ns]
 3   first_coupon_date  202 non-null    datetime64[ns]
 4   mature_dt          225 non-null    datetime64[ns]
 5   isin               225 non-null    string        
 6   issuer_name        225 non-null    string        
 7   coupon_frq         225 non-null    string        
 8   coupon             225 non-null    float64       
 9   tot_issue          225 non-null    float64       
 10  cfi_code           218 non-null    string        
 11  issue_rating       220 non-null    string        
dtypes: datetime64[ns](3), float64(2), string(7)
memory usage: 21.2 KB


In [11]:
df_bonds.head()

,ccy,country,issue_dt,first_coupon_date,mature_dt,isin,issuer_name,coupon_frq,coupon,tot_issue,cfi_code,issue_rating
0,EUR,Netherlands,2009-02-13,2009-07-15,2019-07-15,NL0009086115,STAAT DER NEDERLANDEN,ANNUAL,4.000,5.000000e+09,DBFTFN,AAA
1,NLG,Austria,1994-02-28,1995-02-28,2024-02-28,NL0000133924,AUSTRIA,ANNUAL,6.250,1.000000e+09,DBFTXB,AA+
2,EUR,Netherlands,2012-03-09,2013-01-15,2033-01-15,NL0010071189,STAAT DER NEDERLANDEN,ANNUAL,2.500,4.160000e+09,DBFXXN,AAA
3,USD,United States,2009-05-15,2009-11-15,2019-05-15,US912828KQ20,UNITED STATES TREASURY,SEMI ANNUAL,3.125,6.441100e+10,<NA>,<NA>
4,USD,United States,2010-02-15,2010-08-15,2020-02-15,US912828MP29,UNITED STATES TREASURY,SEMI ANNUAL,3.625,0.000000e+00,<NA>,<NA>


Imputeren ontbrekende waarden voor issue rating. 
Issue rating wordt waar deze ontbreekt ingevuld met de meest voorkomende issue rating voor de issuer.

CFI code wordt waar deze ontbreekt ingevuld met code 'onbekend' = DXXXXX.

Datums met de waarde 1899-12-30 zijn default waardes van het bronsysteem. Deze worden verwijderd.
Daar waar de eerste coupon datum ontbreekt (o.a. zero coupon bonds) - wordt deze aangevuld met de issue datum.
De reden hiervoor is dat we zo zonder veel moeite de looptijd van de bond kunnen berekenen.

In [12]:
df_bonds = make_dataset.impute_bonds(df_bonds)

2022-01-04 15:25:54.382 | INFO     | src.data.make_dataset:impute_bonds:74 - Impute bond data


In [13]:
make_dataset.save_pkl('bonds', df_bonds)

2022-01-04 15:25:54.494 | INFO     | src.data.make_dataset:save_pkl:357 - Save preprocessed bonds data


In [14]:
df_bonds.count()

ccy                  225
country              225
issue_dt             225
first_coupon_date    225
mature_dt            225
isin                 225
issuer_name          225
coupon_frq           225
coupon               225
tot_issue            225
cfi_code             225
issue_rating         225
bond_duration        225
dtype: int64

# Bondprijzen

In [15]:
df_price = make_dataset.get_price()

2022-01-04 15:25:54.726 | INFO     | src.data.make_dataset:get_price:111 - Load bond price data
2022-01-04 15:25:54.727 | INFO     | src.data.make_dataset:read_csv:26 - Loading data from ..\data\raw\price.csv


In [16]:
df_price = make_dataset.impute_price(df_price)

2022-01-04 15:25:55.367 | INFO     | src.data.make_dataset:impute_price:136 - Impute bond price


In [17]:
df_price.head()

,reference_identifier,ccy,rate_dt,mid
0,BE0000332412,EUR,2014-01-20,100.719
1,BE0000332412,EUR,2014-01-24,101.005
2,BE0000332412,EUR,2014-01-28,100.953
3,BE0000332412,EUR,2014-01-22,100.359
4,BE0000332412,EUR,2014-01-21,100.601


In [18]:
df_price.describe()

,mid
count,209276.000000
mean,111.289025
std,15.522802
min,84.429000
25%,102.285000
50%,106.729000
75%,112.926000
max,195.749000


In [19]:
df_price.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 209276 entries, 0 to 220236
Data columns (total 4 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   reference_identifier  209276 non-null  string        
 1   ccy                   209276 non-null  string        
 2   rate_dt               209276 non-null  datetime64[ns]
 3   mid                   209276 non-null  float64       
dtypes: datetime64[ns](1), float64(1), string(2)
memory usage: 8.0 MB


In [20]:
make_dataset.save_pkl('price', df_price)

2022-01-04 15:25:55.976 | INFO     | src.data.make_dataset:save_pkl:357 - Save preprocessed price data


# Government Yield curves

In [21]:
df_yield = make_dataset.get_yield()

2022-01-04 15:25:58.948 | INFO     | src.data.make_dataset:get_yield:144 - Load goverment yield curve data
2022-01-04 15:25:58.949 | INFO     | src.data.make_dataset:read_csv:26 - Loading data from ..\data\raw\yield.csv


In [22]:
df_yield = make_dataset.impute_yield(df_yield)

2022-01-04 15:26:00.298 | INFO     | src.data.make_dataset:impute_yield:175 - Impute yield curve


In [23]:
df_yield.tail()

,country,rate_dt,timeband,ratename,ccy,actual_dt,datedays,bid,offer,int_basis,time
116809,Spain,2021-12-23,5 YEARS,GOV Yield Curve ES BB,EUR,2026-12-23,1831,-0.062,-0.081,ANNUAL,1826 days
116811,Spain,2021-12-23,6 YEARS,GOV Yield Curve ES BB,EUR,2027-12-23,2195,-0.015,-0.030,ANNUAL,2191 days
116812,Spain,2021-12-23,7 YEARS,GOV Yield Curve ES BB,EUR,2028-12-23,2561,0.071,0.057,ANNUAL,2557 days
116813,Spain,2021-12-23,8 YEARS,GOV Yield Curve ES BB,EUR,2029-12-23,2926,0.240,0.229,ANNUAL,2922 days
116815,Spain,2021-12-23,9 YEARS,GOV Yield Curve ES BB,EUR,2030-12-23,3291,0.367,0.357,ANNUAL,3287 days


In [24]:
df_yield.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89750 entries, 1 to 116815
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype          
---  ------     --------------  -----          
 0   country    89750 non-null  object         
 1   rate_dt    89750 non-null  datetime64[ns] 
 2   timeband   89750 non-null  object         
 3   ratename   89750 non-null  string         
 4   ccy        89750 non-null  string         
 5   actual_dt  89750 non-null  datetime64[ns] 
 6   datedays   89750 non-null  int64          
 7   bid        89750 non-null  float64        
 8   offer      89750 non-null  float64        
 9   int_basis  89750 non-null  string         
 10  time       89750 non-null  timedelta64[ns]
dtypes: datetime64[ns](2), float64(2), int64(1), object(2), string(3), timedelta64[ns](1)
memory usage: 8.2+ MB


In [25]:
make_dataset.save_pkl('yield', df_yield)

2022-01-04 15:26:00.940 | INFO     | src.data.make_dataset:save_pkl:357 - Save preprocessed yield data


# Inflation data


In [26]:
df_inflation = make_dataset.get_inflation()  

2022-01-04 15:26:04.447 | INFO     | src.data.make_dataset:get_inflation:195 - Load goverment yield curve data
2022-01-04 15:26:04.448 | INFO     | src.data.make_dataset:read_csv:26 - Loading data from ..\data\raw\DE Inflation.csv
2022-01-04 15:26:04.465 | INFO     | src.data.make_dataset:read_csv:26 - Loading data from ..\data\raw\FR Inflation.csv
2022-01-04 15:26:04.483 | INFO     | src.data.make_dataset:read_csv:26 - Loading data from ..\data\raw\ES Inflation.csv
2022-01-04 15:26:04.499 | INFO     | src.data.make_dataset:read_csv:26 - Loading data from ..\data\raw\IT Inflation.csv
2022-01-04 15:26:04.516 | INFO     | src.data.make_dataset:read_csv:26 - Loading data from ..\data\raw\US Inflation.csv


In [27]:
df_inflation = make_dataset.impute_inflation(df_inflation)


df_inflation.info()

2022-01-04 15:26:18.690 | INFO     | src.data.make_dataset:impute_inflation:223 - Impute inflation curve


<class 'pandas.core.frame.DataFrame'>
Int64Index: 215007 entries, 0 to 216873
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype          
---  ------     --------------   -----          
 0   country    215007 non-null  string         
 1   rate_dt    215007 non-null  datetime64[ns] 
 2   timeband   215007 non-null  string         
 3   inflation  215007 non-null  float64        
 4   ratename   215007 non-null  string         
 5   actual_dt  215007 non-null  datetime64[ns] 
 6   time       215007 non-null  timedelta64[ns]
dtypes: datetime64[ns](2), float64(1), string(3), timedelta64[ns](1)
memory usage: 13.1 MB


In [28]:
df_inflation.head()

,country,rate_dt,timeband,inflation,ratename,actual_dt,time
0,Germany,2021-12-23,1 YEAR,3.28625,Inflation,2022-12-23,365 days
1,Germany,2021-12-22,1 YEAR,3.33875,Inflation,2022-12-22,365 days
2,Germany,2021-12-21,1 YEAR,3.15625,Inflation,2022-12-21,365 days
3,Germany,2021-12-20,1 YEAR,3.01375,Inflation,2022-12-20,365 days
4,Germany,2021-12-17,1 YEAR,2.89875,Inflation,2022-12-17,365 days


In [29]:
make_dataset.save_pkl('inflation', df_inflation)

2022-01-04 15:26:19.353 | INFO     | src.data.make_dataset:save_pkl:357 - Save preprocessed inflation data


In [30]:
df_price

,reference_identifier,ccy,rate_dt,mid
0,BE0000332412,EUR,2014-01-20,100.719
1,BE0000332412,EUR,2014-01-24,101.005
2,BE0000332412,EUR,2014-01-28,100.953
3,BE0000332412,EUR,2014-01-22,100.359
4,BE0000332412,EUR,2014-01-21,100.601
...,...,...,...,...
220232,FR0011486067,EUR,2021-08-31,104.237
220233,FR0011486067,EUR,2021-09-02,104.231
220234,FR0011486067,EUR,2021-10-14,103.889
220235,FR0011486067,EUR,2021-10-15,103.882


In [31]:
df_bp = make_dataset.join_price(df_bonds,df_price )
df_bp = build_features.add_duration(df_bp)

2022-01-04 15:26:27.492 | INFO     | src.features.build_features:add_duration:12 - Add remaining duration...


Om de termspread toe te kunnen voegen moeten we de bond data joinen met de government yield. Hierdoor hebben we minder data beschikbaar.
De vraag is of dit nodig is (of dat het model dit zelf uit kan vogelen)

Hoe meer data we joinen - hoe meer data we kwijt raken. 

In [32]:
df_yield

,country,rate_dt,timeband,ratename,ccy,actual_dt,datedays,bid,offer,int_basis,time
1,France,2016-06-01,1 YEAR,GOV Yield Curve FR BB,EUR,2017-06-01,369,-0.5200,-0.498,ANNUAL,365 days
2,France,2016-06-01,10 YEARS,GOV Yield Curve FR BB,EUR,2026-06-01,3654,0.4755,0.475,ANNUAL,3652 days
3,France,2016-06-01,15 YEARS,GOV Yield Curve FR BB,EUR,2031-06-02,5480,0.4755,0.475,ANNUAL,5479 days
4,France,2016-06-01,2 YEARS,GOV Yield Curve FR BB,EUR,2018-06-01,733,-0.4597,-0.442,ANNUAL,730 days
5,France,2016-06-01,20 YEARS,GOV Yield Curve FR BB,EUR,2036-06-01,7307,1.2151,1.220,ANNUAL,7305 days
...,...,...,...,...,...,...,...,...,...,...,...
116809,Spain,2021-12-23,5 YEARS,GOV Yield Curve ES BB,EUR,2026-12-23,1831,-0.0620,-0.081,ANNUAL,1826 days
116811,Spain,2021-12-23,6 YEARS,GOV Yield Curve ES BB,EUR,2027-12-23,2195,-0.0150,-0.030,ANNUAL,2191 days
116812,Spain,2021-12-23,7 YEARS,GOV Yield Curve ES BB,EUR,2028-12-23,2561,0.0710,0.057,ANNUAL,2557 days
116813,Spain,2021-12-23,8 YEARS,GOV Yield Curve ES BB,EUR,2029-12-23,2926,0.2400,0.229,ANNUAL,2922 days


In [33]:
df_bpy = make_dataset.join_yield(df_bp, df_yield)
df_bpy = build_features.add_term_spread(df_bpy)
df_bpy = build_features.add_bid_offer_spread(df_bpy)
df_bpy.info()

2022-01-04 15:26:28.060 | INFO     | src.features.build_features:add_term_spread:20 - Add term spread...
2022-01-04 15:26:28.064 | INFO     | src.features.build_features:add_bid_offer_spread:31 - Add bid offer spread...


<class 'pandas.core.frame.DataFrame'>
Int64Index: 137019 entries, 0 to 209275
Data columns (total 57 columns):
 #   Column                Non-Null Count  Dtype          
---  ------                --------------  -----          
 0   reference_identifier  88747 non-null  string         
 1   ccy                   88747 non-null  string         
 2   rate_dt               88747 non-null  datetime64[ns] 
 3   mid                   88747 non-null  float64        
 4   country               88747 non-null  object         
 5   issue_dt              88747 non-null  datetime64[ns] 
 6   first_coupon_date     88747 non-null  datetime64[ns] 
 7   mature_dt             88747 non-null  datetime64[ns] 
 8   isin                  88747 non-null  string         
 9   issuer_name           88747 non-null  string         
 10  coupon_frq            88747 non-null  string         
 11  coupon                88747 non-null  float64        
 12  tot_issue             88747 non-null  float64        
 13 

In [34]:
#df_bpiy = make_dataset.fulljoin(df_bonds, df_price, df_inflation, df_yield, df_countryspread)
#df_bpiy = build_features.add_duration(df_bpiy)
#df_bpiy = build_features.add_bid_offer_spread(df_bpiy)

#df_bpy.to_csv(f'../data/processed/bpy.csv')
#df_bpi.to_csv(f'../data/processed/bpi.csv')
#df_bpiy.to_csv(f'../data/processed/bpiy.csv')

In [35]:
#df_pivot1 = df_pivot.drop(['bid','offer'], axis=1, level=0)
#df_pivot1.columns = df_pivot1.columns.droplevel(1)
#df_pivot1.reset_index()

In [36]:
#df_bpiy

All in one make statement...

In [37]:
make_dataset.make_data()

2022-01-04 15:26:28.808 | INFO     | src.data.make_dataset:get_bond_data:41 - Load bond data
2022-01-04 15:26:28.810 | INFO     | src.data.make_dataset:read_csv:26 - Loading data from ..\data\raw\bonds.csv
2022-01-04 15:26:28.876 | INFO     | src.data.make_dataset:impute_bonds:74 - Impute bond data
2022-01-04 15:26:28.882 | INFO     | src.data.make_dataset:save_pkl:357 - Save preprocessed bonds data
2022-01-04 15:26:28.901 | INFO     | src.data.make_dataset:get_price:111 - Load bond price data
2022-01-04 15:26:28.902 | INFO     | src.data.make_dataset:read_csv:26 - Loading data from ..\data\raw\price.csv
2022-01-04 15:26:29.429 | INFO     | src.data.make_dataset:impute_price:136 - Impute bond price
2022-01-04 15:26:29.457 | INFO     | src.data.make_dataset:save_pkl:357 - Save preprocessed price data
2022-01-04 15:26:32.591 | INFO     | src.data.make_dataset:get_yield:144 - Load goverment yield curve data
2022-01-04 15:26:32.592 | INFO     | src.data.make_dataset:read_csv:26 - Loading d

In [47]:
df_bp.columns

Index(['reference_identifier', 'ccy', 'rate_dt', 'mid', 'country', 'issue_dt',
       'first_coupon_date', 'mature_dt', 'isin', 'issuer_name', 'coupon_frq',
       'coupon', 'tot_issue', 'cfi_code', 'issue_rating', 'bond_duration',
       'remain_duration'],
      dtype='object')

In [46]:
df = build_features.encode_coupon_freq(df_bp)


[autoreload of src.features.build_features failed: Traceback (most recent call last):
  File "C:\Users\M\AppData\Local\pypoetry\Cache\virtualenvs\timeseries-NkPdx04N-py3.8\lib\site-packages\IPython\extensions\autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\M\AppData\Local\pypoetry\Cache\virtualenvs\timeseries-NkPdx04N-py3.8\lib\site-packages\IPython\extensions\autoreload.py", line 394, in superreload
    module = reload(module)
  File "C:\Users\M\.pyenv\pyenv-win\versions\3.8.10\lib\imp.py", line 314, in reload
    return importlib.reload(module)
  File "C:\Users\M\.pyenv\pyenv-win\versions\3.8.10\lib\importlib\__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 604, in _exec
  File "<frozen importlib._bootstrap_external>", line 848, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "c:\Users\M\SkyDrive\Documents\GitHub\BondTr

AttributeError: module 'src.features.build_features' has no attribute 'encode_coupon_freq'